**4.创建Xgboost模型**

**4.1.将上述训练集切分为训练集好测试集**

In [1]:
import sys
import time
import operator
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from matplotlib import pylab as plt
from datetime import datetime
from sklearn.model_selection import GridSearchCV
pd.set_option('display.max_rows', 4)

In [2]:
data = pd.read_csv('data/result_训练集.csv')
data 

,用户ID,商品ID,商品类别,3天内商品-行为总数_1.0,3天内商品-行为总数_2.0,3天内商品-行为总数_3.0,3天内商品-行为总数_4.0,3天内商品-行为总数_5.0,3天内商品-行为总数_6.0,3天内商品类别-行为总数_1.0,...,商品类别-收藏行为均值,商品类别-点击行为均值,存在差评,差评率,评论数量_0,评论数量_1,评论数量_2,评论数量_3,评论数量_4,标签
0,202633.0,12564.0,8.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,...,20.866667,5167.400000,1.0,0.0260,0.0,0.0,0.0,0.0,1.0,1.0
1,218498.0,149854.0,8.0,4.0,0.0,0.0,0.0,0.0,4.0,6.0,...,20.866667,5167.400000,1.0,0.0403,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14617,255327.0,36416.0,5.0,2.0,0.0,0.0,0.0,0.0,3.0,12.0,...,97.333333,19536.333333,0.0,0.0000,0.0,0.0,1.0,0.0,0.0,0.0
14618,239228.0,10598.0,9.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,46.866667,10623.100000,1.0,0.0343,0.0,0.0,0.0,0.0,1.0,0.0


In [3]:
data.columns #打印列索引

Index(['用户ID', '商品ID', '商品类别', '3天内商品-行为总数_1.0', '3天内商品-行为总数_2.0',
       '3天内商品-行为总数_3.0', '3天内商品-行为总数_4.0', '3天内商品-行为总数_5.0', '3天内商品-行为总数_6.0',
       '3天内商品类别-行为总数_1.0',
       ...
       '商品类别-收藏行为均值', '商品类别-点击行为均值', '存在差评', '差评率', '评论数量_0', '评论数量_1',
       '评论数量_2', '评论数量_3', '评论数量_4', '标签'],
      dtype='object', length=293)

In [4]:
data_x = data.loc[:,data.columns != '标签']#提取特征数据集
data_y = data.loc[:,data.columns == '标签']#提取标签数据集

展示特征数据集

In [5]:
data_x

,用户ID,商品ID,商品类别,3天内商品-行为总数_1.0,3天内商品-行为总数_2.0,3天内商品-行为总数_3.0,3天内商品-行为总数_4.0,3天内商品-行为总数_5.0,3天内商品-行为总数_6.0,3天内商品类别-行为总数_1.0,...,商品类别-购买行为均值,商品类别-收藏行为均值,商品类别-点击行为均值,存在差评,差评率,评论数量_0,评论数量_1,评论数量_2,评论数量_3,评论数量_4
0,202633.0,12564.0,8.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,...,8.400000,20.866667,5167.400000,1.0,0.0260,0.0,0.0,0.0,0.0,1.0
1,218498.0,149854.0,8.0,4.0,0.0,0.0,0.0,0.0,4.0,6.0,...,8.400000,20.866667,5167.400000,1.0,0.0403,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14617,255327.0,36416.0,5.0,2.0,0.0,0.0,0.0,0.0,3.0,12.0,...,58.966667,97.333333,19536.333333,0.0,0.0000,0.0,0.0,1.0,0.0,0.0
14618,239228.0,10598.0,9.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,15.833333,46.866667,10623.100000,1.0,0.0343,0.0,0.0,0.0,0.0,1.0


展示标签数据集

In [6]:
data_y

,标签
0,1.0
1,1.0
...,...
14617,0.0
14618,0.0


切分训练集和测试集

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data_x,data_y,test_size = 0.25, random_state = 0)

In [8]:
x_test.shape

(3655, 292)

再将测试集拆分为验证集和测试集

In [9]:
x_val = x_test.iloc[:1500,:] 
y_val = y_test.iloc[:1500,:] 
x_test = x_test.iloc[1500:,:]  
y_test = y_test.iloc[1500:,:] 

print (x_val.shape)
print (x_test.shape) 

(1500, 292)
(2155, 292)


展示训练集特征

In [10]:
x_train#展示训练集的数据

,用户ID,商品ID,商品类别,3天内商品-行为总数_1.0,3天内商品-行为总数_2.0,3天内商品-行为总数_3.0,3天内商品-行为总数_4.0,3天内商品-行为总数_5.0,3天内商品-行为总数_6.0,3天内商品类别-行为总数_1.0,...,商品类别-购买行为均值,商品类别-收藏行为均值,商品类别-点击行为均值,存在差评,差评率,评论数量_0,评论数量_1,评论数量_2,评论数量_3,评论数量_4
9861,265828.0,131542.0,4.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0,...,52.833333,119.266667,34685.733333,1.0,0.0199,0.0,0.0,0.0,0.0,1.0
3833,282219.0,13334.0,8.0,2.0,0.0,0.0,0.0,0.0,2.0,10.0,...,18.200000,82.800000,17920.700000,1.0,0.0336,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10799,220721.0,135272.0,4.0,1.0,0.0,0.0,0.0,0.0,2.0,28.0,...,55.866667,127.100000,36895.200000,1.0,0.0237,0.0,0.0,0.0,0.0,1.0
2732,243286.0,26229.0,9.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,...,3.866667,16.000000,3543.566667,1.0,0.0202,0.0,0.0,0.0,0.0,1.0


删除无用的两列用户ID和商品Id

In [11]:
del x_train['用户ID']
del x_train['商品ID']
del x_val['用户ID']
del x_val['商品ID'] 
x_train

,商品类别,3天内商品-行为总数_1.0,3天内商品-行为总数_2.0,3天内商品-行为总数_3.0,3天内商品-行为总数_4.0,3天内商品-行为总数_5.0,3天内商品-行为总数_6.0,3天内商品类别-行为总数_1.0,3天内商品类别-行为总数_2.0,3天内商品类别-行为总数_3.0,...,商品类别-购买行为均值,商品类别-收藏行为均值,商品类别-点击行为均值,存在差评,差评率,评论数量_0,评论数量_1,评论数量_2,评论数量_3,评论数量_4
9861,4.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0,0.0,0.0,...,52.833333,119.266667,34685.733333,1.0,0.0199,0.0,0.0,0.0,0.0,1.0
3833,8.0,2.0,0.0,0.0,0.0,0.0,2.0,10.0,0.0,0.0,...,18.200000,82.800000,17920.700000,1.0,0.0336,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10799,4.0,1.0,0.0,0.0,0.0,0.0,2.0,28.0,0.0,0.0,...,55.866667,127.100000,36895.200000,1.0,0.0237,0.0,0.0,0.0,0.0,1.0
2732,9.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,...,3.866667,16.000000,3543.566667,1.0,0.0202,0.0,0.0,0.0,0.0,1.0


将数据转成DMatrix的格式

In [12]:
#xgboost模型需要把当前的结果转成DMatrix的格式，xbg矩阵赋值
dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_val, label=y_val) 

In [13]:
#参数设置
#'eval_metric':'auc'：设置对有效数据的度量方法，auc代表曲线下面积
parameters_of_model = {'n_estimators': 4000, 'max_depth': 3, 'eval_metric':'auc'} 

**4.2.训练模型**

In [14]:
n_estimators = parameters_of_model['n_estimators']#设置迭代计算次数
evallist = [(dtrain, 'train'), (dvalid, 'eval')]#设置一个列表，训练过程中评估列表中的元素，便于观察训练过程中验证集的效果

#进行xgboost训练，dtrain是训练数据，由xgb.DMatrix传入：early_stopping_rounds是早期停止次数
model_xgb = xgb.train(parameters_of_model, dtrain, n_estimators, evallist, early_stopping_rounds=10) 
model_xgb.save_model('data/result_model_xgb.model') 

[17:10:26] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.93047	eval-auc:0.92669
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 10 rounds.
[1]	train-auc:0.94280	eval-auc:0.93759
[2]	train-auc:0.95550	eval-auc:0.94948
[3]	train-auc:0.95857	eval-auc:0.95434
[4]	train-auc:0.96126	eval-auc:0.95722
[5]	train-auc:0.96197	eval-auc:0.95900
[6]	train-auc:0.96349	eval-auc:0.95963
[7]	train-auc:0.96368	eval-auc:0.95952
[8]	train-auc:0.96534	eval-auc:0.96111
[9]	train-auc:0.96580	eval-auc:0.96165
[10]	train-auc:0.96728	eval-auc:0.96218
[11]	train-auc:0.96771	eval-auc:0.96110

上述结果解释：最好的一次迭代，在第17次，train-auc:0.97159（训练集上的模型评价指标）；eval-auc:0.96325（验证集上的模型评估指标）。<br/>
early_stopping_rounds是早期停止次数，这里设为10，验证集的精确度迭代到一定程度在10次内不能再继续提高，就停止迭代。

In [15]:
print (model_xgb.attributes())#打印最好的结果

{'best_iteration': '17', 'best_msg': '[17]\ttrain-auc:0.97159\teval-auc:0.96325', 'best_score': '0.963255'}


**4.3.特征重要程度排序和显示**

定义create_feature_map函数，将特征放在feature列中

In [16]:
def create_feature_map(features):
    outfile = open(r'data/result_xgb.fmap', 'w',encoding='utf-8')#以只读方式打开文件data/result_xgb.fmap，‘w’打开一个文件用于写入
    i = 0
    for feat in features:
        #将特征名称写入文件，使用format函数，将序号值和特征名称放在前面的{}中，用\t(制表符)，\n(换行符)分隔
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1 
    outfile.close()

features = list(x_train.columns[:])#传入特征列名参数
create_feature_map(features) #调用create_feature_map函数，执行这个操作

定义feature_importance函数，计算特征重要程度

In [17]:
def feature_importance(model_xgb):
    importance = model_xgb.get_fscore(fmap=r'data/result_xgb.fmap')#使用xgb.get_fscore，获取各特征分数
    
    #将各特征按照其对应的重要程度从小到大的顺序排列
    #importance.items()设置字典键值对，key：value，operator，itemgetter(1)代表按照第二个元素的次序对元组进行排序，reverse=True代表逆序
    importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
    
    df = pd.DataFrame(importance, columns=['feature', 'fscore'])#将“特征”和“重要性”转换成DataFrame的形式，设置列名
    df['fscore'] = df['fscore'] / df['fscore'].sum()#计算各特征分数在特征总分数中的占比，保存到“fscore”中
    
    df.to_csv('data/result_feature_importance.csv',index=False)#保将文件存成csv文件

feature_importance(model_xgb)#调用feature_importance函数，执行这个操作

展示结果

In [18]:
fi = pd.read_csv('data/result_feature_importance.csv')#读取保存的文件
fi

,feature,fscore
0,商品品牌,0.108808
1,属性2_1,0.056995
...,...,...
84,购买/浏览转化率_y,0.005181
85,7天内商品类别-行为总数_6.0,0.005181


**4.4.模型评估**

查看测试数据

In [19]:
x_test.head()

,用户ID,商品ID,商品类别,3天内商品-行为总数_1.0,3天内商品-行为总数_2.0,3天内商品-行为总数_3.0,3天内商品-行为总数_4.0,3天内商品-行为总数_5.0,3天内商品-行为总数_6.0,3天内商品类别-行为总数_1.0,...,商品类别-购买行为均值,商品类别-收藏行为均值,商品类别-点击行为均值,存在差评,差评率,评论数量_0,评论数量_1,评论数量_2,评论数量_3,评论数量_4
6765,261684.0,70839.0,8.0,6.0,0.0,0.0,0.0,1.0,8.0,6.0,...,25.733333,116.800000,24942.466667,1.0,0.0272,0.0,0.0,0.0,0.0,1.0
13767,255257.0,142477.0,8.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,...,45.466667,190.266667,41133.200000,1.0,0.0285,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9116,215915.0,71940.0,6.0,2.0,0.0,0.0,0.0,0.0,2.0,4.0,...,34.800000,81.333333,16189.166667,1.0,0.0222,0.0,0.0,0.0,0.0,1.0
10055,263738.0,50181.0,8.0,2.0,0.0,0.0,0.0,0.0,4.0,8.0,...,35.233333,149.266667,32299.033333,1.0,0.0685,0.0,0.0,0.0,0.0,1.0


评估模型，进行预测

In [20]:
users = x_test[['用户ID', '商品ID', '商品类别']].copy()
del x_test['商品ID']#删除“用户ID”列
x_test_DMatrix = xgb.DMatrix(x_test)#删除“商品ID”列 
y_predict = model_xgb.predict(x_test_DMatrix, ntree_limit=model_xgb.best_ntree_limit)#进行测试

In [21]:
x_test['predict_label'] = y_predict#把预测出来的结果合并到x_test里
x_test

,商品类别,3天内商品-行为总数_1.0,3天内商品-行为总数_2.0,3天内商品-行为总数_3.0,3天内商品-行为总数_4.0,3天内商品-行为总数_5.0,3天内商品-行为总数_6.0,3天内商品类别-行为总数_1.0,3天内商品类别-行为总数_2.0,3天内商品类别-行为总数_3.0,...,商品类别-收藏行为均值,商品类别-点击行为均值,存在差评,差评率,评论数量_0,评论数量_1,评论数量_2,评论数量_3,评论数量_4,predict_label
6765,8.0,6.0,0.0,0.0,0.0,1.0,8.0,6.0,0.0,0.0,...,116.800000,24942.466667,1.0,0.0272,0.0,0.0,0.0,0.0,1.0,0.244716
13767,8.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,...,190.266667,41133.200000,1.0,0.0285,0.0,0.0,0.0,0.0,1.0,0.229104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,4.0,4.0,1.0,1.0,0.0,0.0,3.0,17.0,3.0,3.0,...,52.133333,16111.900000,1.0,0.0263,0.0,0.0,0.0,0.0,1.0,-0.014281
6367,7.0,1.0,0.0,0.0,0.0,0.0,1.0,14.0,0.0,0.0,...,36.166667,9447.266667,1.0,0.0432,0.0,0.0,0.0,0.0,1.0,0.019120


定义label函数，把predict_label列转化为0或1的形式

In [22]:
def label(column):
    
    if column['predict_label'] > 0.5:
        column['predict_label'] = 1
    else:
    
        column['predict_label'] = 0
    return column

x_test = x_test.apply(label,axis = 1)
x_test     

,商品类别,3天内商品-行为总数_1.0,3天内商品-行为总数_2.0,3天内商品-行为总数_3.0,3天内商品-行为总数_4.0,3天内商品-行为总数_5.0,3天内商品-行为总数_6.0,3天内商品类别-行为总数_1.0,3天内商品类别-行为总数_2.0,3天内商品类别-行为总数_3.0,...,商品类别-收藏行为均值,商品类别-点击行为均值,存在差评,差评率,评论数量_0,评论数量_1,评论数量_2,评论数量_3,评论数量_4,predict_label
6765,8.0,6.0,0.0,0.0,0.0,1.0,8.0,6.0,0.0,0.0,...,116.800000,24942.466667,1.0,0.0272,0.0,0.0,0.0,0.0,1.0,0.0
13767,8.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,...,190.266667,41133.200000,1.0,0.0285,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,4.0,4.0,1.0,1.0,0.0,0.0,3.0,17.0,3.0,3.0,...,52.133333,16111.900000,1.0,0.0263,0.0,0.0,0.0,0.0,1.0,0.0
6367,7.0,1.0,0.0,0.0,0.0,0.0,1.0,14.0,0.0,0.0,...,36.166667,9447.266667,1.0,0.0432,0.0,0.0,0.0,0.0,1.0,0.0


把用户真是的购买情况的标签列合并到表里

In [23]:
x_test['true_label'] = y_test
x_test

,商品类别,3天内商品-行为总数_1.0,3天内商品-行为总数_2.0,3天内商品-行为总数_3.0,3天内商品-行为总数_4.0,3天内商品-行为总数_5.0,3天内商品-行为总数_6.0,3天内商品类别-行为总数_1.0,3天内商品类别-行为总数_2.0,3天内商品类别-行为总数_3.0,...,商品类别-点击行为均值,存在差评,差评率,评论数量_0,评论数量_1,评论数量_2,评论数量_3,评论数量_4,predict_label,true_label
6765,8.0,6.0,0.0,0.0,0.0,1.0,8.0,6.0,0.0,0.0,...,24942.466667,1.0,0.0272,0.0,0.0,0.0,0.0,1.0,0.0,0.0
13767,8.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,...,41133.200000,1.0,0.0285,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,4.0,4.0,1.0,1.0,0.0,0.0,3.0,17.0,3.0,3.0,...,16111.900000,1.0,0.0263,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6367,7.0,1.0,0.0,0.0,0.0,0.0,1.0,14.0,0.0,0.0,...,9447.266667,1.0,0.0432,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [24]:
#把“用户ID”和“商品ID”两列添加回表中
x_test['用户ID'] = users['用户ID']
x_test['商品ID'] = users['商品ID']
x_test

,商品类别,3天内商品-行为总数_1.0,3天内商品-行为总数_2.0,3天内商品-行为总数_3.0,3天内商品-行为总数_4.0,3天内商品-行为总数_5.0,3天内商品-行为总数_6.0,3天内商品类别-行为总数_1.0,3天内商品类别-行为总数_2.0,3天内商品类别-行为总数_3.0,...,差评率,评论数量_0,评论数量_1,评论数量_2,评论数量_3,评论数量_4,predict_label,true_label,用户ID,商品ID
6765,8.0,6.0,0.0,0.0,0.0,1.0,8.0,6.0,0.0,0.0,...,0.0272,0.0,0.0,0.0,0.0,1.0,0.0,0.0,261684.0,70839.0
13767,8.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,...,0.0285,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255257.0,142477.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,4.0,4.0,1.0,1.0,0.0,0.0,3.0,17.0,3.0,3.0,...,0.0263,0.0,0.0,0.0,0.0,1.0,0.0,0.0,270571.0,6468.0
6367,7.0,1.0,0.0,0.0,0.0,0.0,1.0,14.0,0.0,0.0,...,0.0432,0.0,0.0,0.0,0.0,1.0,0.0,0.0,272061.0,2378.0


获取所有真实购买用户、所有预测购买的用户、所用真实购买的用户-商品组合以及预测会购买的用户-商品组合

In [25]:
all_user_set = x_test[x_test['true_label']==1]['用户ID'].unique()#获取所有真实购买用户数据，unique()代表去重
print (len(all_user_set))#打印所有真实购买用户数据

all_user_test_set = x_test[x_test['predict_label'] == 1]['用户ID'].unique()#获取所有预测会存在购买行为的用户数据
print (len(all_user_test_set))#打印所有预测会存在购买行为的用户数据

#获取真实购买数据中用户-商品组合
#使用map(str)函数“用户ID”和“商品ID”转换为字符串的形式
all_user_item_pair = x_test[x_test['true_label']==1]['用户ID'].map(str) + '-' + x_test[x_test['true_label']==1]['商品ID'].map(str)
all_user_item_pair = np.array(all_user_item_pair)#转换成数组的格式
print (len(all_user_item_pair))#打印所有存在购买行为的用户-商品组合数量

#获取所有预测会存在购买行为的用户-商品组合数据
#使用map(str)函数“用户ID”和“商品ID”转换为字符串的形式
all_user_test_item_pair = x_test[x_test['predict_label'] == 1]['用户ID'].map(str) + '-' + x_test[x_test['predict_label'] == 1]['商品ID'].map(str)
all_user_test_item_pair = np.array(all_user_test_item_pair)#转换成数组格式
print (len(all_user_test_item_pair))#打印所有预测会存在购买行为的用户-商品组合数量

189
152
220
172


计算预测用户购买的准确率和召回率

In [26]:
#设置一个for循环，统计预测正确的预测不正确的用户数量
true, false = 0,0
for 商品ID in all_user_test_set:
    
    #如果预测的所有购买行为的用户，确实存在购买行为，则true值加1
    if 商品ID  in all_user_set:
        true += 1
    else:
    #如果预测的有购买行为的用户，不存在购买行为，则false值加1
        false += 1
#计算预测购买用户的准确率和召回率    
all_user_accuracy = 1.0 * true / ( true + false)#准确率 = 预测正确的存在购买行为的用户数量/预测的所有的用户数量
all_user_recall = 1.0 * true / len(all_user_set)#召回率 = 预测正确存在购买行为的用户数量/所有真实的存在购买行为的用户数据
print ('所有用户中预测存在购买行为的用户的准确率为 ' + str(all_user_accuracy))
print ('所有用户中预测存在购买行为的用户的召回率为 ' + str(all_user_recall))

所有用户中预测存在购买行为的用户的准确率为 0.8289473684210527
所有用户中预测存在购买行为的用户的召回率为 0.6666666666666666


计算预测的用户-商品组合的准确率和召回率

In [27]:
#设置一个for循环，统计预测正确和预测不正确的数量
true, false = 0,0
for user_item_pair in all_user_test_item_pair:
    if user_item_pair in all_user_item_pair:
        
        #如果预测的用户-商品组合，为真实的用户-商品组合，则true值加1
        true += 1
    else:
       #如果预测的用户-商品组合，为预测错误的用户-商品组合，则neg值加1
        false += 1

#计算预测购买用户的准确率和召回率
all_item_accuracy = 1.0 * true / ( true + false)#准确率 = 预测正确的存在购买行为的用户数量/预测的所有的用户数量
all_item_recall = 1.0 * true / len(all_user_item_pair)#召回率 = 预测正确存在购买行为的用户数量/所有真实的存在购买行为的用户数据
print ('所有用户中预测购买商品的准确率为 ' + str(all_item_accuracy))
print ('所有用户中预测购买商品的召回率为 ' + str(all_item_recall))

#计算F11和F12两个指标
F11 = 6.0 * all_user_recall * all_user_accuracy / (5.0 * all_user_recall + all_user_accuracy)
F12 = 5.0 * all_item_accuracy * all_item_recall / (2.0 * all_item_recall + 3 * all_item_accuracy)
score = 0.4 * F11 + 0.6 * F12
print ('F11=' + str(F11))
print ('F12=' + str(F12))
print ('score=' + str(score))

所有用户中预测购买商品的准确率为 0.8430232558139535
所有用户中预测购买商品的召回率为 0.6590909090909091
F11=0.796628029504742
F12=0.7221115537848606
score=0.7519181440728131
